In [19]:
def stalta_pick(stream, stalen, ltalen, trig_on, trig_off, freqmin=False,
                freqmax=False, debug=0, show=False):
    """
    Basic sta/lta picker, suggest using alternative in obspy.
    Simple sta-lta (short-term average/long-term average) picker, using \
    obspy's stalta routine to generate the characteristic function.

    Currently very basic quick wrapper, there are many other (better) options \
    in obspy, found \
    `here <http://docs.obspy.org/packages/autogen/obspy.signal.trigger.html>`_.

    :type stream: obspy.Stream
    :param stream: The stream to pick on, can be any number of channels.
    :type stalen: float
    :param stalen: Length of the short-term average window in seconds.
    :type ltalen: float
    :param ltalen: Length of the long-term average window in seconds.
    :type trig_on: float
    :param trig_on: sta/lta ratio to trigger a detection/pick
    :type trig_off: float
    :param trig_off: sta/lta ratio to turn the trigger off - no further picks\
        will be made between exceeding trig_on until trig_off is reached.
    :type freqmin: float
    :param freqmin: Low-cut frequency in Hz for bandpass filter
    :type freqmax: float
    :param freqmax: High-cut frequency in Hz for bandpass filter
    :type debug: int
    :param debug: Debug output level from 0-5.
    :type show: bool
    :param show: Show picks on waveform.

    :returns: obspy.core.event.Event

    .. rubric:: Example

    >>> from obspy import read
    >>> from eqcorrscan.utils.picker import stalta_pick
    >>> st = read()
    >>> event = stalta_pick(st, stalen=0.2, ltalen=4, trig_on=10,
    ...             trig_off=1, freqmin=3.0, freqmax=20.0)
    >>> event.creation_info.author
    'EQcorrscan'
    """
    from obspy.signal.trigger import classic_sta_lta, trigger_onset
    from obspy.signal.trigger import plot_trigger
    from obspy import UTCDateTime
    from obspy.core.event import Event, Pick, WaveformStreamID
    from obspy.core.event import CreationInfo, Comment, Origin
    #import eqcorrscan.utils.plotting as plotting

    event = Event()
    event.origins.append(Origin())
    event.creation_info = CreationInfo(author='EQcorrscan',
                                       creation_time=UTCDateTime())
    event.comments.append(Comment(text='stalta'))
    picks = []
    for tr in stream:
        # We are going to assume, for now, that if the pick is made on the
        # horizontal channel then it is an S, otherwise we will assume it is
        # a P-phase: obviously a bad assumption...
        if tr.stats.channel[-1] == 'Z':
            phase = 'P'
        else:
            phase = 'S'
        if freqmin and freqmax:
            tr.detrend('simple')
            tr.filter('bandpass', freqmin=freqmin, freqmax=freqmax,
                      corners=3, zerophase=True)
        df = tr.stats.sampling_rate
        cft = classic_sta_lta(tr.data, int(stalen * df), int(ltalen * df))
        if debug > 3:
            plot_trigger(tr, cft, trig_on, trig_off)
        triggers = trigger_onset(cft, trig_on, trig_off)
        for trigger in triggers:
            on = tr.stats.starttime + (trigger[0] / df)
            # off = tr.stats.starttime + (trigger[1] / df)
            wav_id = WaveformStreamID(station_code=tr.stats.station,
                                      channel_code=tr.stats.channel,
                                      network_code=tr.stats.network)
            pick = Pick(waveform_id=wav_id, phase_hint=phase, time=on)
            if debug > 2:
                print('Pick made:')
                print(pick)
            picks.append(pick)
    # QC picks
    #del pick
    pick_stations = list(set([pick.waveform_id.station_code for pick in picks]))
    for pick_station in pick_stations:
        station_picks = [pick for pick in picks if
                         pick.waveform_id.station_code == pick_station]
        # If P-pick is after S-picks, remove it.
        p_time = [pick.time for pick in station_picks if pick.phase_hint == 'P']
        s_time = [pick.time for pick in station_picks if pick.phase_hint == 'S']
        #if p_time > s_time:
        #    p_pick = [pick for pick in station_picks if pick.phase_hint == 'P']
        #    for pick in p_pick:
        #        print('P pick after S pick, removing P pick')
        #        picks.remove(pick)
    if show:
        plotting.pretty_template_plot(stream, picks=picks, title='Autopicks',
                                      size=(8, 9))
    event.picks = picks
    event.origins[0].time = min([pick.time for pick in event.picks]) - 1
    #event.origins[0].latitude = float('nan')
    #event.origins[0].longitude = float('nan')
    # Set arbitrary origin time
    return event

In [28]:
import obspy
from obspy.core import read
filename = '/home/felipe/sources/DADOS/seiscomp_NRT/2022/NB/NBTA/HHZ.D/NB.NBTA..HHZ.D.2022.115'
st = read(filename)

In [29]:
stalta_pick(st,0.5,10000,20,5,freqmin=2,freqmax=15)

Event:	2022-04-25T17:14:19.155000Z | None, None

	   resource_id: ResourceIdentifier(id="smi:local/38c041ae-1d88-4bbf-88d7-0562c0909eef")
	 creation_info: CreationInfo(author='EQcorrscan', creation_time=UTCDateTime(2022, 4, 26, 15, 18, 46, 215952))
	          ---------
	      comments: 1 Elements
	         picks: 2 Elements
	       origins: 1 Elements

In [12]:
st

1 Trace(s) in Stream:
NB.NBTA..HHZ | 2022-04-25T00:00:04.135000Z - 2022-04-26T00:00:01.525000Z | 100.0 Hz, 8639740 samples